For an introduction on EM algorithm see "*What is the expectation maximization algorithm?*" paper by **Chuong B Do & Serafim Batzoglou**:

https://ai.stanford.edu/~chuongdo/papers/em_tutorial.pdf

here you find a simple implementation of the example shown in that tutorial:

In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
NTOSSES=20 # number of tosses of each coin
NSETS=10 # number of experiments in which we toss a coin NTOSSES times

In [3]:
# probability of heads of each coin - this is not observed!
coins={'A' : 0.3,\
       'B' : 0.7}

In [4]:
# randomly choose a coin NSET times (with equal probability)
z=np.random.choice(list(coins.keys()), NSETS, p=(0.5,0.5))

In [5]:
z

array(['B', 'B', 'B', 'A', 'B', 'B', 'A', 'B', 'B', 'B'], dtype='<U1')

In [6]:
# toss each coin in z NTOSSES times
x=np.array([np.random.binomial(1, coins[i], NTOSSES) for i in z])

In [7]:
x

array([[1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1],
       [1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0],
       [1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0],
       [0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1],
       [0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0],
       [1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1],
       [0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1],
       [1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1],
       [0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1],
       [0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1]])

In [8]:
# estimate the parameter of each coin *knowing* the coin used
thetaA=np.sum(x[z=='A',:])/(np.sum((z=='A').astype(np.int)) * NTOSSES)
thetaB=np.sum(x[z=='B',:])/(np.sum((z=='B').astype(np.int)) * NTOSSES)

In [9]:
print(thetaA, thetaB)

0.375 0.74375


### Expectation maximization

Now, suppose we don't know $z$. We only know that each coin is was drawn with equal probability. We only get to observe $x$.

let us guess some inital values for theta:

In [10]:
guessThetaA=0.4
guessThetaB=0.6

Now we compute the probability of each coin (as specified by these guess parameters) given the observation (x).

Rember that:

\begin{equation}
p(\text{coin}=A|\text{observation}=x)=\frac{p(\text{observation}=x|\text{coin}=A)p(coin=A)}{p(\text{observation}=x|\text{coin}=A)p(coin=A) + p(\text{observation}=x|\text{coin}=B)p(coin=B)}
\end{equation}

Note that $p(\text{observation}=x|\text{coin}=A)$ is just a binomial with parameter `guessThetaA`. In addition, $p(coin=A)$ is half as specified previously.

The same can be computed for $p(\text{coin}=B|\text{observation}=x)$

In [11]:
pXgivenA=guessThetaA**np.sum(x, axis=1)*(1-guessThetaA)**np.sum(1.-x, axis=1)
pXgivenB=guessThetaB**np.sum(x, axis=1)*(1-guessThetaB)**np.sum(1.-x, axis=1)
pAgivenX=(pXgivenA*0.5)/(pXgivenA*0.5+pXgivenB*0.5)
pBgivenX=(pXgivenB*0.5)/(pXgivenA*0.5+pXgivenB*0.5)

Now we compute the expected numbers of heads and tails according to this distribution:

In [12]:
CoinA_heads=np.sum(x, axis=1)*pAgivenX
CoinA_tails=np.sum(1-x,axis=1)*pAgivenX
CoinB_heads=np.sum(x, axis=1)*pBgivenX
CoinB_tails=np.sum(1-x,axis=1)*pBgivenX

With these expected values, we can compute new parameters:

In [13]:
guessThetaA=np.sum(CoinA_heads)/(np.sum(CoinA_heads)+np.sum(CoinA_tails))
guessThetaB=np.sum(CoinB_heads)/(np.sum(CoinB_heads)+np.sum(CoinB_tails))

Iterate:

In [14]:
for i in range(100):
    pXgivenA=guessThetaA**np.sum(x, axis=1)*(1-guessThetaA)**np.sum(1.-x, axis=1)
    pXgivenB=guessThetaB**np.sum(x, axis=1)*(1-guessThetaB)**np.sum(1.-x, axis=1)
    pAgivenX=(pXgivenA*0.5)/(pXgivenA*0.5+pXgivenB*0.5)
    pBgivenX=(pXgivenB*0.5)/(pXgivenA*0.5+pXgivenB*0.5)
    
    CoinA_heads=np.sum(x, axis=1)*pAgivenX
    CoinA_tails=np.sum(1-x,axis=1)*pAgivenX
    CoinB_heads=np.sum(x, axis=1)*pBgivenX
    CoinB_tails=np.sum(1-x,axis=1)*pBgivenX
    
    guessThetaA=np.sum(CoinA_heads)/(np.sum(CoinA_heads)+np.sum(CoinA_tails))
    guessThetaB=np.sum(CoinB_heads)/(np.sum(CoinB_heads)+np.sum(CoinB_tails))

In [15]:
print(guessThetaA,guessThetaB)

0.441741912054951 0.7529957227878182
